<a href="https://colab.research.google.com/github/flparen/gif3001-socketapi/blob/master/GIF3001_API_Socket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API socket
L'API socket a été introduit dans BSD UNIX 4.1 en 1981. L'API offre une abstraction des fonctions d'émission et de réception de données. 

Un **socket** est une interface de communication utilisé par un processus utilisateur. Cette interface prend la forme d'un descripteur de fichier, et est utilisé pour la transmission et réception de données.


![alt text](https://github.com/flparen/gif3001-socketapi/raw/master/socket_interface.png)



La documentation officielle de l'API socket de Python 3 est disponible à l'adresse https://docs.python.org/3/library/socket.html



Le tableau suivant résume quelques fonctions offertent par l'API socket de Python. Ces fontions utilisent les fonctions correspondantes du système d'exploitation. Nous allons examiner l'utilisation de ces fonctions dans le contexte d'une application client/serveur TCP et UDP.

|**Fonction**|**Description**|**client?**|**serveur?**|
|:-----------|--------------|:---------:|:----------:|
|socket()|Crée un socket|oui|oui|
|connect()|Initie une connexion (TCP)|oui|non|
|bind()|Lie le socket sur un adresse IP locale|opt|oui|
|listen()|Place le socket en mode écoute pour recevoir des connexions client|non|oui|
|accept()|Accepte une connexion client|non|oui|
|recv(), send(), ...|Reçoit et envoie des données|oui|oui|
|close()|Ferme le socket|oui|oui|



Il existe plusieurs familles de socket. Ceux que nous allons utiliser ici sont les sockets de type **IPv4 et IPv6**.

|**Famille**|**Description**|
|:---------:|:-------------:|
|socket.AF\_INET|IPv4|
|socket.AF\_INET6|IPv6|
|socket.AF\_UNIX|Communication locale|
|socket.AF\_PACKET|Utilisé pour la transmission et réception de packet à la couche liaison (_paquet bas-niveau_)|

Python utilise une paire (hôte, port) pour désigner une adresse socket IPv4. *hôte* désigne l'adresse IPv4 ou le nom de la machine, alors que *port* indique le numéro de port du service (ex: 80 pour HTTP).

Pour IPv6, l'adresse socket est représenté par (hôte, port, flowinfo, scopeid), où *hôte* et l'adresse IPv6 ou le nom de la machine, et *port* est le numéro de port du service (semblable à IPv4).


Pour utiliser l'interface socket en Python, il est nécessaire d'importer le module `socket`
```python
import socket
```



## Client TCP

Le schéma suivant démontre l'interaction entre un client et serveur TCP du point de vue de l'API socket.

![alt text](https://github.com/flparen/gif3001-socketapi/raw/master/client_serveur_TCP.png)

Regardons l'implémentation du client en Python


In [ ]:
import socket

SERVEUR = "chat.gif3001.beon.ca"
PORT = 3001

s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)

L'objet socket est créé. Un socket de type TCP doit être **connecté** pour pouvoir transmettre des données. La fonction *connect()* indique au système d'exploitation d'établir une connexion TCP vers le serveur et le numéro de port indiqué en paramètre.

In [ ]:
s.connect((SERVEUR, PORT))

L'objet socket est maintenant connecté. Nous pouvons utiliser la fonction *send()* pour transmettre des données. Les données doivent être de type *octet* (bytes)

In [ ]:
s.send("Test en classe".encode())

In [ ]:
data = s.recv(1024)

In [ ]:
print (data)

In [ ]:
s.close()

## Client UDP

Le schéma suivant démontre l'interaction entre un client et serveur UDP.

![alt text](https://github.com/flparen/gif3001-socketapi/raw/master/client_serveur_UDP.png)

Du côté client, la fonction ```connect()``` n'est pas utilisé car UDP est un protocole **sans connexion**. Le code client doit alors spécifier l'adresse et le numéro de port pour chaque données transmises au socket via la fonction ```sendto()``` 

Les données reçues sur le socket UDP sont lues par l'application via la fonction ```recvfrom()```. En plus de retourner les données reçues, cette fonction retourne également l'adresse et le numéro de port identifiant l'envoyeur des données. Cette information est importante afin de pouvoir transmettre une réponse, le cas échéant.

In [ ]:
import socket

SERVEUR = "chat.gif3001.beon.ca"
PORT = 3001

s = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)

In [ ]:
s.sendto("Test en classe".encode(), (SERVEUR,PORT))

In [ ]:
data, adresse = s.recvfrom(1024)

In [ ]:
print (data)

In [ ]:
print (adresse)

In [ ]:
s.close()

## Fonctions de résolution de nom et adresse

### getaddrinfo()
```getaddrinfo(host, port, family=0, type=0, proto=0, flags=0)``` est utilisé pour obtenir une liste d'adresse IP et numéro de port à partir d'un **nom de machine** (ex: www.ulaval.ca) et du **nom d'un service** (ex: http). La résolution d'adresse DNS est utilisé par le système d'exploitation pour obtenir les adresses IP.

```getaddrinfo()``` retourne un tuple de 5 éléments: ```(family, type, proto, canonname, sockaddr)``` Ces éléments offrent toutes les informations nécessaires pour créer un socket et établir la connexion vers le service identifié. 

* ```family, type, proto``` sont les informations nécessaires pour la fonction ```socket()```
* ```sockaddr``` est l'adresse socket utilisée pour les fonctions ```connect()``` ou ```sendto()``` 



In [ ]:
import socket

socket.getaddrinfo("www.google.com", "https", type=socket.SOCK_STREAM)

### getnameinfo()

```getnameinfo(sockaddr, flags)``` permet l'opération inverse de getaddrinfo(), c'est à dire retourner le nom de la machine et le nom du service à partir d'une adresse socket IPv4 ou IPv6. La résolution d'adresse DNS est utilisé par le système d'exploitation pour obtenir le nom correspondant à l'adresse IP. Si le nom n'existe pas, l'adresse IP est retournée.

Plus d'information est disponible sur cette fonction avec la commande ```man getnameinfo``` sur un poste Linux.


In [ ]:
import socket

socket.getnameinfo(('132.203.244.29', 25), 0)

In [ ]:
socket.getnameinfo(('2620:0:1af0:dc00::2', 80), 0)